In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score

import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")




In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import joblib

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold,KFold
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score
import os

from datasets import concatenate_datasets,load_dataset,load_from_disk
from transformers import AutoModel, AutoTokenizer, AdamW, DataCollatorWithPadding

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pytl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import polars as pl
from torch_geometric.nn import GCNConv,SAGEConv


In [3]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)

In [4]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

train['dataset_mode'] = 0
valid['dataset_mode'] = 1
test['dataset_mode'] = 2

data = pd.concat([train,valid,test]).reset_index(drop = True)



In [5]:
data = pd.read_feather('all_feat.feather')



In [6]:



drop_feat =['tfidf_autherName'] + ['PID','label','autherID','pid_title','autherName', 'abstract', 'keywords', 'venue', 'authors', 'index','cos_sim_pid','autherName_count'] 
used_feat = [f for f in data.columns if f not in (['PID','label','autherID','dataset_mode'] + drop_feat )]
print(len(used_feat))
print(used_feat)
ycol = 'label'

#del data
gc.collect()

372
['bge_m3_pid_title_sim_mean', 'bge_m3_pid_title_sim_max', 'bge_m3_pid_title_sim_min', 'bge_m3_pid_title_sim_std', 'bge_m3_pid_title_sim_median', 'bge_m3_pid_abstract_sim_mean', 'bge_m3_pid_abstract_sim_max', 'bge_m3_pid_abstract_sim_min', 'bge_m3_pid_abstract_sim_std', 'bge_m3_pid_abstract_sim_median', 'tfidf_pid_abstract_sim_mean', 'tfidf_pid_abstract_sim_max', 'tfidf_pid_abstract_sim_min', 'tfidf_pid_abstract_sim_std', 'tfidf_pid_abstract_sim_median', 'tfidf_pid_title_sim_mean', 'tfidf_pid_title_sim_max', 'tfidf_pid_title_sim_min', 'tfidf_pid_title_sim_std', 'tfidf_pid_title_sim_median', 'tfidf_pid_venue_sim_mean', 'tfidf_pid_venue_sim_max', 'tfidf_pid_venue_sim_min', 'tfidf_pid_venue_sim_std', 'tfidf_pid_venue_sim_median', 'tfidf_pid_keyword_text_sim_mean', 'tfidf_pid_keyword_text_sim_max', 'tfidf_pid_keyword_text_sim_min', 'tfidf_pid_keyword_text_sim_std', 'tfidf_pid_keyword_text_sim_median', 'jaccard_pid_keyword_cnt_1_mean', 'jaccard_pid_keyword_cnt_1_max', 'jaccard_pid_keywor

0

In [7]:
df_oof = pd.read_feather('oof2/cat_oof.feather')
del df_oof['label_prob']

temp =  pd.read_feather('oof2/lgb_oof.feather')
df_oof['lgb_prob'] = temp['label_prob']

temp =  pd.read_feather('oof2/xgb_oof.feather')
df_oof['xgb_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/cat_oof.feather')
df_oof['cat_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/feat_select_lgb_oof.feather')
df_oof['lgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_xgb_oof.feather')
df_oof['xgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_cat_oof.feather')
df_oof['cat_prob_2'] = temp['label_prob']

prediction = pd.read_feather('oof2/cat_prediction.feather')
del prediction['label_prob']

temp =  pd.read_feather('oof2/lgb_prediction.feather')
prediction['lgb_prob'] = temp['label_prob']

temp =  pd.read_feather('oof2/xgb_prediction.feather')
prediction['xgb_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/cat_prediction.feather')
prediction['cat_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/feat_select_lgb_prediction.feather')
prediction['lgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_xgb_prediction.feather')
prediction['xgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_cat_prediction.feather')
prediction['cat_prob_2'] = temp['label_prob']
temp = pd.concat([df_oof,prediction]).reset_index(drop = True)

"""
temp[[ 'lgb_prob', 'xgb_prob', 'cat_prob',
       'lgb_prob_2', 'xgb_prob_2', 'cat_prob_2']] =  temp.groupby('autherID')[[ 'lgb_prob', 'xgb_prob', 'cat_prob',
       'lgb_prob_2', 'xgb_prob_2', 'cat_prob_2']].transform('rank') / temp.groupby('autherID')[[ 'lgb_prob', 'xgb_prob', 'cat_prob',
       'lgb_prob_2', 'xgb_prob_2', 'cat_prob_2']].transform('count')
"""
del temp['label']

In [9]:
data = data.merge(temp,on  =['autherID','PID'],how = 'left')

In [10]:
data = pd.concat([data[['PID', 'label', 'autherID', 'autherName', 'dataset_mode','lgb_prob', 'xgb_prob', 'cat_prob', 'lgb_prob_2',
       'xgb_prob_2', 'cat_prob_2']],(data[used_feat ] - data[used_feat ].min()) / (data[used_feat].max()  - data[used_feat ].min())],axis = 1)

In [11]:
for f in tqdm(used_feat):
    data[f] = data[f].fillna(data[f].mean())

100%|██████████| 372/372 [00:00<00:00, 484.36it/s]


In [12]:
"""
node_encoder = joblib.load('ProNE/node_encoder.pkl')
node_decoder = joblib.load('ProNE/node_decoder.pkl')
edges = pd.read_csv('ProNE/edges_list.txt',sep = ' ',header = None)

edges[0] = edges[0].map(node_decoder)
edges[1] = edges[1].map(node_decoder)
"""
edges = joblib.load('edges_all.pkl')
data['id'] = data.index

In [12]:
data

,PID,label,autherID,autherName,dataset_mode,lgb_prob,xgb_prob,cat_prob,lgb_prob_2,xgb_prob_2,...,title_emb_svd_10,title_emb_svd_11,title_emb_svd_12,title_emb_svd_13,title_emb_svd_14,title_emb_svd_15,year,author_rank_ratio,author_rank,id
0,YzOCpPTO,0.0,Iki037dt,atsushi ochiai,0,0.995569,0.998563,0.994757,0.995349,0.999189,...,0.513112,0.403550,0.462179,0.519085,0.374388,0.472013,0.993083,0.284541,0.000956,0
1,AblgcGjH,0.0,Iki037dt,atsushi ochiai,0,0.961333,0.985927,0.978535,0.947156,0.978245,...,0.334536,0.602430,0.409886,0.526043,0.647095,0.257562,0.993577,0.373974,0.000637,1
2,B5aouLse,0.0,Iki037dt,atsushi ochiai,0,0.955576,0.980310,0.983442,0.966846,0.991403,...,0.462084,0.533263,0.384081,0.414022,0.523506,0.454810,0.988636,0.777413,0.001912,2
3,u1G7wBEv,0.0,Iki037dt,atsushi ochiai,0,0.984241,0.994281,0.994938,0.987069,0.996917,...,0.408238,0.575191,0.552440,0.385124,0.197069,0.337199,0.981719,1.000000,0.002869,3
4,W7w6P8lA,0.0,Iki037dt,atsushi ochiai,0,0.996644,0.998641,0.997534,0.996980,0.999352,...,0.400295,0.662736,0.534882,0.404532,0.483754,0.316045,0.995553,0.649425,0.001753,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326795,vhwIyDJm,NaN,Pu6wvwLr,yubo chen,2,0.941757,0.993945,0.972270,0.973219,0.991588,...,0.426105,0.422989,0.542385,0.514842,0.401795,0.466348,0.999012,0.888706,0.002231,326795
326796,wXUkX3Go,NaN,Pu6wvwLr,yubo chen,2,0.963601,0.991254,0.984374,0.952115,0.995302,...,0.503675,0.559642,0.343830,0.612504,0.412387,0.523680,0.998024,0.749589,0.000637,326796
326797,yJLWNkt6,NaN,Pu6wvwLr,yubo chen,2,0.864842,0.977128,0.937601,0.864351,0.986817,...,0.297019,0.287015,0.365777,0.446811,0.536170,0.634783,0.998518,0.499179,0.000956,326797
326798,yZpKg3wN,NaN,Pu6wvwLr,yubo chen,2,0.914313,0.981711,0.957200,0.957060,0.983163,...,0.515214,0.272920,0.676037,0.358894,0.473250,0.498873,0.996047,0.198686,0.000000,326798


In [13]:
df = data[['autherID']].drop_duplicates().reset_index(drop = True)

In [14]:
data

,PID,label,autherID,autherName,dataset_mode,lgb_prob,xgb_prob,cat_prob,lgb_prob_2,xgb_prob_2,...,title_emb_svd_10,title_emb_svd_11,title_emb_svd_12,title_emb_svd_13,title_emb_svd_14,title_emb_svd_15,year,author_rank_ratio,author_rank,id
0,YzOCpPTO,0.0,Iki037dt,atsushi ochiai,0,0.995569,0.998563,0.994757,0.995349,0.999189,...,0.513112,0.403550,0.462179,0.519085,0.374388,0.472013,0.993083,0.284541,0.000956,0
1,AblgcGjH,0.0,Iki037dt,atsushi ochiai,0,0.961333,0.985927,0.978535,0.947156,0.978245,...,0.334536,0.602430,0.409886,0.526043,0.647095,0.257562,0.993577,0.373974,0.000637,1
2,B5aouLse,0.0,Iki037dt,atsushi ochiai,0,0.955576,0.980310,0.983442,0.966846,0.991403,...,0.462084,0.533263,0.384081,0.414022,0.523506,0.454810,0.988636,0.777413,0.001912,2
3,u1G7wBEv,0.0,Iki037dt,atsushi ochiai,0,0.984241,0.994281,0.994938,0.987069,0.996917,...,0.408238,0.575191,0.552440,0.385124,0.197069,0.337199,0.981719,1.000000,0.002869,3
4,W7w6P8lA,0.0,Iki037dt,atsushi ochiai,0,0.996644,0.998641,0.997534,0.996980,0.999352,...,0.400295,0.662736,0.534882,0.404532,0.483754,0.316045,0.995553,0.649425,0.001753,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326795,vhwIyDJm,NaN,Pu6wvwLr,yubo chen,2,0.941757,0.993945,0.972270,0.973219,0.991588,...,0.426105,0.422989,0.542385,0.514842,0.401795,0.466348,0.999012,0.888706,0.002231,326795
326796,wXUkX3Go,NaN,Pu6wvwLr,yubo chen,2,0.963601,0.991254,0.984374,0.952115,0.995302,...,0.503675,0.559642,0.343830,0.612504,0.412387,0.523680,0.998024,0.749589,0.000637,326796
326797,yJLWNkt6,NaN,Pu6wvwLr,yubo chen,2,0.864842,0.977128,0.937601,0.864351,0.986817,...,0.297019,0.287015,0.365777,0.446811,0.536170,0.634783,0.998518,0.499179,0.000956,326797
326798,yZpKg3wN,NaN,Pu6wvwLr,yubo chen,2,0.914313,0.981711,0.957200,0.957060,0.983163,...,0.515214,0.272920,0.676037,0.358894,0.473250,0.498873,0.996047,0.198686,0.000000,326798


In [15]:

idx = 18
feats= dict()
id_ = df.loc[idx]['autherID']
temp = data[data['autherID'] == id_].reset_index(drop = True)
temp_edges = edges[(edges[0]>=temp['id'].min()) & (edges[0]<temp['id'].max())]
temp_edges[0] = temp_edges[0] - temp['id'].min()
temp_edges[1] = temp_edges[1] - temp['id'].min()

temp = temp.sample(frac = 1).reset_index(drop = True)
t_dict = dict(zip((temp['id'] -temp['id'].min()).to_list(),temp.index.to_list()))
temp_edges[0] = temp_edges[0].map(t_dict)
temp_edges[1] = temp_edges[1].map(t_dict)
temp_edges['e0'] = temp_edges[[0,1]].min(axis = 1)
temp_edges['e1'] = temp_edges[[0,1]].max(axis = 1)
temp_edges = temp_edges.sort_values(['e0','e1']).reset_index(drop = True)


In [14]:

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df,data,used_feat, edges,targets ='label',mode ='Test'):
        
        self.df = df.copy()
        self.data = data.copy()
        
        self.used_feat = used_feat
        self.edges = edges
        self.targets = targets
        self.mode = mode

        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        df = dict()
        id_ = self.df.loc[idx]['autherID']
        temp = self.data[self.data['autherID'] == id_].reset_index(drop = True)
        temp_edges = self.edges[(self.edges[0]>=temp['id'].min()) & (self.edges[0]<temp['id'].max())]
        temp_edges[0] = temp_edges[0] - temp['id'].min()
        temp_edges[1] = temp_edges[1] - temp['id'].min()
        #if self.mode == 'Train':
        temp = temp.sample(frac = 1).reset_index(drop = True)
        t_dict = dict(zip((temp['id'] -temp['id'].min()).to_list(),temp.index.to_list()))
        temp_edges[0] = temp_edges[0].map(t_dict)
        temp_edges[1] = temp_edges[1].map(t_dict)
        temp_edges['e0'] = temp_edges[[0,1]].min(axis = 1)
        temp_edges['e1'] = temp_edges[[0,1]].max(axis = 1)
        temp_edges = temp_edges.sort_values(['e0','e1']).reset_index(drop = True)



        df['node_feat'] = torch.tensor(temp[self.used_feat].values, dtype=torch.float)
        df['edge_index'] = torch.tensor(temp_edges[['e0','e1']].values, dtype=torch.long).T
        df['id'] =  torch.tensor(temp['id'])
        df['sp_feats'] = torch.tensor(temp[['lgb_prob', 'xgb_prob', 'cat_prob', 'lgb_prob_2',
       'xgb_prob_2', 'cat_prob_2']].values, dtype=torch.float)
        if self.mode == 'Train':
            df['targets'] = torch.tensor(temp[self.targets])
        #print(123)
        return df

In [15]:
#torch.tensor(temp_edges[[0,1]].values, dtype=torch.long).T.shape

In [16]:
class cfg:
    #num_features = ['action_time','cursor_position','word_count'] + ['down_time_minute', 'down_time_hour','time_diff']
    #cat_features = ['activity','text_change']
    targets = 'label'
    exp_name = 'exp1'
    used_feat = used_feat
    
    seed = 42
    
    batch_size = 1
    val_batch_size = 1
    lr = 3e-5
    num_workers = 4
    prefetch = 4
    epochs = 10

In [17]:
#data[data['fold_id']<8].reset_index(drop = True)

In [18]:
#temp = pd.read_feather('nn/kfold_fold.feather')
#set(temp[temp['fold_id']<8]['autherID'])

In [19]:
data_train = data[data['label'].notna()].reset_index(drop = True)
data_test = data[data['label'].isna()].reset_index(drop = True)

#data_train = data[data['autherID'].isin(set(temp[temp['fold_id']<8]['autherID']))].reset_index(drop = True)
#data_test = data[~data['autherID'].isin(set(temp[temp['fold_id']<8]['autherID']))].reset_index(drop = True)

data_train['label'] = 1 - data_train['label']


df_train = data_train[['autherID']].drop_duplicates().reset_index(drop = True)
df_test = data_test[['autherID']].drop_duplicates().reset_index(drop = True)


In [22]:
temp = pd.read_feather('nn/kfold_fold.feather')
df_train = df_train.merge(temp[['autherID','fold_id']].drop_duplicates(subset = ['autherID'],keep = 'first'),on = ['autherID'],how = 'left')

In [20]:

folds = 5
kfold = KFold(n_splits=folds, shuffle=True, random_state=cfg.seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train)):
    df_train.loc[val_idx,f'fold_id'] = fold_id


In [24]:

train_dataset = Dataset(df_train[df_train['fold_id']!=0].reset_index(drop = True),data_train, cfg.used_feat,edges, cfg.targets, 'Train' )
val_dataset = Dataset(df_train[df_train['fold_id']==0].reset_index(drop = True),data_train, cfg.used_feat,edges, cfg.targets, 'Train' )

train_dataloader = DataLoader(train_dataset,
                              batch_size=cfg.batch_size,
                              #sampler=train_sampler,
                              drop_last=True,
                              pin_memory=True,
                              num_workers=cfg.num_workers,
                              prefetch_factor=cfg.prefetch)
val_dataloader = DataLoader(val_dataset,
                            batch_size=cfg.val_batch_size,
                            #sampler=val_sampler,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=cfg.num_workers,
                            prefetch_factor=cfg.prefetch)

test_dataset = Dataset(df_test,data_test, cfg.used_feat,edges, cfg.targets, 'test' )

test_dataloader = DataLoader(test_dataset,
                            batch_size=cfg.val_batch_size,
                            #sampler=val_sampler,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=cfg.num_workers,
                            prefetch_factor=cfg.prefetch)


In [25]:
for f in train_dataloader:
    break

In [27]:
node_dense = torch.nn.Sequential(nn.Linear(1, 32),
                                          nn.ReLU(),
                                          nn.Linear(32, 32),
                                          nn.ReLU(),
                                          nn.Linear(32, 32),
                                          nn.ReLU(),
                                    )

convs = [SAGEConv(256,128) for i in range(3)]
rev_convs = [SAGEConv(256,128) for i in range(3)]

In [26]:
from sklearn.metrics import roc_auc_score


In [27]:
f.keys()

dict_keys(['node_feat', 'edge_index', 'id', 'sp_feats', 'targets'])

In [21]:

def rmse_loss(y_pred, target):
    loss =  (y_pred - target).abs() * (y_pred - target).abs() 
    return loss.mean()**0.5


class OAG_GNN_Prediction_Model(pytl.LightningModule):
    def __init__(
        self,
        used_feat,
        dropout=0.5,
        lr=3e-4,
        
    ):
        super().__init__()

        self.save_hyperparameters()
        self.lr = lr
        self.dropout = dropout
        
        self.convs =  nn.ModuleList([SAGEConv(16+16,8),SAGEConv(16+16,8),SAGEConv(16+16,8)])
        self.rev_convs =  nn.ModuleList([SAGEConv(16+16,8),SAGEConv(16+16,8),SAGEConv(16+16,8)])
       

        self.node_dense = torch.nn.Sequential(nn.Linear(372, 256),
                                          nn.LeakyReLU(),
                                          nn.Linear(256, 64),
                                          nn.LeakyReLU(),
                                          nn.Linear(64, 16),
                                          nn.LeakyReLU(),
                                    )
        
        self.node_dense_2 = torch.nn.Sequential(nn.Linear(6, 16),
                                          nn.LeakyReLU(),
                                          nn.Linear(16, 16),
                                          nn.LeakyReLU(),

                                    )
        
        self.MLP = nn.Sequential(
                            nn.Linear(16+16  ,32 )
                            ,nn.LeakyReLU()
                            ,nn.Linear(32  ,32 )
                            ,nn.LeakyReLU()
                            ,nn.Linear(32,1)
                            ,nn.Sigmoid()
                            )
        
        self.loss_function = nn.SmoothL1Loss(reduction='mean') 

    def forward(self, inputs):
        
        edge_index = inputs['edge_index'][0]
        rev_edge_index = torch.flip(inputs['edge_index'][0], (0,))


        X1 = self.node_dense(inputs['node_feat'])
        X2 = self.node_dense_2(inputs['sp_feats'])
        
        
        X = torch.cat([X1,X2],dim = -1)

        
        
        
        for i in range(1):
            X = torch.cat(
                [
                    X2,
                    self.convs[i](X, edge_index),
                    self.rev_convs[i](X, rev_edge_index),
                ],
                dim=-1,
            )
            X = nn.LeakyReLU()(X)
        
        
        X = self.MLP(X)
        return X[0]

    def training_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        #print(y_hat,y.float())
        #print(y_hat_.shape,y.shape)

        loss = nn.BCELoss()(y_hat_,y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        #print(y_hat_.shape,y.shape)
        loss = nn.BCELoss()(y_hat_, y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())



        
        self.log("valid_loss", loss)
        
        return {'val_loss': loss, 'logits': y_hat_,'targets':y}   

    def validation_epoch_end(self,batch):
        avg_loss = torch.stack([x['val_loss'] for x in batch]).mean()
        avg_score = np.mean([roc_auc_score(x['targets'].cpu().detach().numpy(),x['logits'].cpu().detach().numpy(),) for x in batch])

        
        print(f'epoch {trainer.current_epoch} validation loss {avg_loss}')
        print(f'epoch {trainer.current_epoch} validation score {avg_score}')
        
        return {'val_loss': avg_loss,'val_mcrmse':avg_score}
    
    
    
    def test_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        loss =nn.BCELoss()(y_hat_, y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())
        
        self.log("test_loss", loss)
        
        return y_hat



    def configure_optimizers(self):
        #optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        #    optimizer, patience=2, factor=0.5
        #)
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_training_steps=10000, num_warmup_steps= 0,
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]
    
    """
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.cfg.optimizer.lr)
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_training_steps=self.trainer.max_steps, **self.cfg.scheduler
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]
    """

In [22]:
"""
def rmse_loss(y_pred, target):
    loss =  (y_pred - target).abs() * (y_pred - target).abs() 
    return loss.mean()**0.5


class OAG_GNN_Prediction_Model(pytl.LightningModule):
    def __init__(
        self,
        used_feat,
        dropout=0.5,
        lr=4e-5,
        
    ):
        super().__init__()

        self.save_hyperparameters()
        self.lr = lr
        self.dropout = dropout
        
        self.convs =  nn.ModuleList([SAGEConv(256+1,128),SAGEConv(256+1,128),SAGEConv(256+1,128)])
        self.rev_convs =  nn.ModuleList([SAGEConv(256+1,128),SAGEConv(256+1,128),SAGEConv(256+1,128)])
        

        self.node_dense = torch.nn.Sequential(nn.Linear(320+1, 1024),
                                          nn.GELU(),
                                          nn.Linear(1024, 1024),
                                          nn.GELU(),
                                          nn.Linear(1024, 256),
                                          nn.GELU(),
                                    )
        
        self.MLP = nn.Sequential(
                            nn.Linear(256+1  ,512 )
                            ,nn.GELU()
                            ,nn.Linear(512  ,64 )
                            ,nn.GELU()
                            ,nn.Linear(64,1)
                            ,nn.Sigmoid()
                            )
        
        self.loss_function = nn.SmoothL1Loss(reduction='mean') 

    def forward(self, inputs):
        
        edge_index = inputs['edge_index'][0]
        rev_edge_index = torch.flip(inputs['edge_index'][0], (0,))

        X = torch.cat([inputs['node_feat'],inputs['sp_feats']],dim = -1)
        X = self.node_dense(X)
        X = torch.cat([X,inputs['sp_feats']],dim = -1)

        
        
        
        for i in range(3):
            X = torch.cat(
                [
                    inputs['sp_feats'],
                    self.convs[i](X, edge_index),
                    self.rev_convs[i](X, rev_edge_index),
                ],
                dim=-1,
            )
        
        
        X = self.MLP(X)
        return X[0]

    def training_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        #print(y_hat,y.float())
        #print(y_hat_.shape,y.shape)

        loss = nn.BCELoss()(y_hat_,y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        #print(y_hat_.shape,y.shape)
        loss = nn.BCELoss()(y_hat_, y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())



        
        self.log("valid_loss", loss)
        
        return {'val_loss': loss, 'logits': y_hat_,'targets':y}   

    def validation_epoch_end(self,batch):
        avg_loss = torch.stack([x['val_loss'] for x in batch]).mean()
        avg_score = np.mean([roc_auc_score(x['targets'].cpu().detach().numpy(),x['logits'].cpu().detach().numpy(),) for x in batch])

        
        print(f'epoch {trainer.current_epoch} validation loss {avg_loss}')
        print(f'epoch {trainer.current_epoch} validation score {avg_score}')
        
        return {'val_loss': avg_loss,'val_mcrmse':avg_score}
    
    
    
    def test_step(self, batch, batch_idx):
        trg_out = batch['targets']
        y_hat = self(batch)

        y_hat_ = y_hat.view(-1)
        y = trg_out.view(-1)
        loss =nn.BCELoss()(y_hat_, y.float())
        #loss  = nn.CrossEntropyLoss()(y_hat_,y.float())
        
        self.log("test_loss", loss)
        
        return y_hat



    def configure_optimizers(self):
        #optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        #    optimizer, patience=2, factor=0.5
        #)
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_training_steps=10000, num_warmup_steps= 0,
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]
    
    """



'\ndef rmse_loss(y_pred, target):\n    loss =  (y_pred - target).abs() * (y_pred - target).abs() \n    return loss.mean()**0.5\n\n\nclass OAG_GNN_Prediction_Model(pytl.LightningModule):\n    def __init__(\n        self,\n        used_feat,\n        dropout=0.5,\n        lr=4e-5,\n        \n    ):\n        super().__init__()\n\n        self.save_hyperparameters()\n        self.lr = lr\n        self.dropout = dropout\n        \n        self.convs =  nn.ModuleList([SAGEConv(256+1,128),SAGEConv(256+1,128),SAGEConv(256+1,128)])\n        self.rev_convs =  nn.ModuleList([SAGEConv(256+1,128),SAGEConv(256+1,128),SAGEConv(256+1,128)])\n        \n\n        self.node_dense = torch.nn.Sequential(nn.Linear(320+1, 1024),\n                                          nn.GELU(),\n                                          nn.Linear(1024, 1024),\n                                          nn.GELU(),\n                                          nn.Linear(1024, 256),\n                                          nn

In [23]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar,
    StochasticWeightAveraging,
    EarlyStopping,
)
from pytorch_lightning.loggers import WandbLogger
from transformers import get_cosine_schedule_with_warmup


In [24]:
123

123

In [25]:
oof = []
sub = []
for fold_id in range(5):
    
    train_dataset = Dataset(df_train[df_train['fold_id']!=fold_id].reset_index(drop = True),data_train, cfg.used_feat,edges, cfg.targets, 'Train' )
    val_dataset = Dataset(df_train[df_train['fold_id']==fold_id].reset_index(drop = True),data_train, cfg.used_feat,edges, cfg.targets, 'Train' )

    train_dataloader = DataLoader(train_dataset,
                                  batch_size=cfg.batch_size,
                                  #sampler=train_sampler,
                                  drop_last=True,
                                  pin_memory=True,
                                  num_workers=cfg.num_workers,
                                  prefetch_factor=cfg.prefetch)
    val_dataloader = DataLoader(val_dataset,
                                batch_size=cfg.val_batch_size,
                                #sampler=val_sampler,
                                drop_last=False,
                                pin_memory=True,
                                num_workers=cfg.num_workers,
                                prefetch_factor=cfg.prefetch)
    
    test_dataset = Dataset(df_test,data_test, cfg.used_feat,edges, cfg.targets, 'test' )

    test_dataloader = DataLoader(test_dataset,
                                batch_size=cfg.val_batch_size,
                                #sampler=val_sampler,
                                drop_last=False,
                                pin_memory=True,
                                num_workers=cfg.num_workers,
                                prefetch_factor=cfg.prefetch)

    seed_everything(cfg.seed)
    checkpoint_cb = ModelCheckpoint(
        verbose=True,
        monitor='valid_loss',
        mode='min',
        save_top_k=1,
        save_last=False,)

    lr_monitor = LearningRateMonitor("epoch")
    progress_bar = RichProgressBar()
    model_summary = RichModelSummary(max_depth=2)
    """
    pl_logger = WandbLogger(
        name=cfg.exp_name,
        project="kaggle_lwp",
    )
    pl_logger.log_hyperparams(cfg)
    """

    model = OAG_GNN_Prediction_Model(cfg.used_feat)


    EarlyStopping_callbacl = EarlyStopping(monitor="valid_loss", min_delta=0.00, patience=3, verbose= True)


    trainer = pytl.Trainer( precision=32,max_epochs=20,devices=1,accelerator="gpu",callbacks=[checkpoint_cb, lr_monitor, progress_bar, model_summary,EarlyStopping_callbacl])
    trainer.fit(model, train_dataloader, val_dataloader,)

    
    pred_list = []
    id_list = []
    label_list = []
    sp_list = []
    for i,f in tqdm(enumerate(val_dataloader)):
        label_list += list(f['targets'].numpy()[0])
        id_list += list(f['id'].numpy()[0])
        pred_list += list(model(f).detach().numpy().T[0])
        sp_list += list(f['sp_feats'].numpy()[0].T[0])
        
    df_oof = pd.DataFrame(id_list,columns = ['id'])
    df_oof['label_prob'] = pred_list
    df_oof['label'] = label_list
    df_oof['lgb_prob'] = sp_list
    
    
    oof.append(df_oof.copy())
    
    
    pred_list = []
    id_list = []
    label_list = []
    sp_list = []
    for i,f in tqdm(enumerate(test_dataloader)):
        id_list += list(f['id'].numpy()[0])
        pred_list += list(model(f).detach().numpy().T[0])
        sp_list += list(f['sp_feats'].numpy()[0].T[0])
        
        
    prediction = pd.DataFrame(id_list,columns = ['id'])
    prediction['label_prob'] = pred_list
    prediction['lgb_prob'] = sp_list
    
    sub.append(prediction.copy())
    
    del model
    gc.collect()

Global seed set to 42
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name           ┃ Type         ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ convs          │ ModuleList   │  1.6 K │
│ 1  │ convs.0        │ SAGEConv     │    520 │
│ 2  │ convs.1        │ SAGEConv     │    520 │
│ 3  │ convs.2        │ SAGEConv     │    520 │
│ 4  │ rev_convs      │ ModuleList   │  1.6 K │
│ 5  │ rev_convs.0    │ SAGEConv     │    520 │
│ 6  │ rev_convs.1    │ SAGEConv     │    520 │
│ 7  │ rev_convs.2    │ SAGEConv     │    520 │
│ 8  │ node_dense     │ Sequential   │  112 K │
│ 9  │ node_dense.0   │ Linear       │ 95.5 K │
│ 10 │ node_dense.1   │ LeakyReLU    │      0 │
│ 11 │ node_dense.2   │ Linear       │ 16.4 K │
│ 12 │ node_dense.3   │ LeakyReLU    │      0 │
│ 13 │ node_dense.4   │ Linear       │  1.0 K │
│ 14 │ node_dense.5   │ LeakyReLU    │      0 │
│ 15 │ node_dense_2   │ Sequential   │    384 │
│ 16 │ node_dense_2.0 │ Linear       │    112 │
│ 17 │ node_dense_2.1 │ LeakyReLU    │      0 │
│ 18 │ node_dense_2.2 │ Linear       │    272 │
│ 19 │ node_dense_2.3 │ LeakyReLU    │      0 │
│ 20 │ MLP            │ Sequential   │  2.1 K │
│ 21 │ MLP.0          │ Linear       │  1.1 K │
│ 22 │ MLP.1          │ LeakyReLU    │      0 │
│ 23 │ MLP.2          │ Linear       │  1.1 K │
│ 24 │ MLP.3          │ LeakyReLU    │      0 │
│ 25 │ MLP.4          │ Linear       │     33 │
│ 26 │ MLP.5          │ Sigmoid      │      0 │
│ 27 │ loss_function  │ SmoothL1Loss │      0 │
└────┴────────────────┴──────────────┴────────┘

Trainable params: 118 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 118 K                                                                                                
Total estimated model params size (MB): 0

Output()

epoch 0 validation loss 0.6549386978149414

...

Epoch 13, global step 8736: 'valid_loss' was not in top 1


epoch 14 validation loss 0.36289745569229126

epoch 14 validation score 0.8429763136827912

Monitored metric valid_loss did not improve in the last 3 records. Best score: 0.362. Signaling Trainer to stop.
Epoch 14, global step 9360: 'valid_loss' was not in top 1


155it [00:02, 67.21it/s]
885it [00:10, 83.31it/s]


In [26]:
df_oof = pd.concat(oof).reset_index(drop = True).sort_values('id').reset_index(drop = True)
prediction = sub[0].sort_values('id').reset_index(drop = True)
prediction['label_prob'] = 0
prediction['lgb_prob'] = 0
for i in range(5):
    prediction['label_prob'] += (sub[i].sort_values('id').reset_index(drop = True)['label_prob'] / 5)
    prediction['lgb_prob'] += (sub[i].sort_values('id').reset_index(drop = True)['lgb_prob'] / 5)


In [27]:
df_oof =  df_oof.merge(data[['autherID','id','PID','dataset_mode']],on = 'id',how = 'left')
prediction =  prediction.merge(data[['autherID','id','PID','dataset_mode']],on = 'id',how = 'left')


In [28]:
df_oof.to_feather('oof2/gnn_oof.feather')
prediction.to_feather('oof2/gnn_prediction.feather')